<a href="https://colab.research.google.com/github/JulienHelfenstein/World_model/blob/main/02_train_vae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Définir le chemin racine de votre projet
PROJECT_ROOT = "/content/drive/My Drive/Colab Notebooks/World_model"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import numpy as np
import os
from tqdm import tqdm

In [ ]:
# --- 1. Configuration et Hyperparamètres ---
DATA_FILE = os.path.join(PROJECT_ROOT, "data/carracing_data.npz")
MODEL_SAVE_PATH = os.path.join(PROJECT_ROOT, "vae.pth")
z_dim = 32          # Dimension de l'espace latent (doit correspondre au RNN)
image_channels = 3  # RGB
learning_rate = 1e-3
batch_size = 64     # Augmentez si vous avez plus de VRAM, diminuez si vous manquez de mémoire
num_epochs = 10     # 10 époques est un début. 30-50 est mieux si vous avez le temps.

In [ ]:
# --- 2. Le Dataset Personnalisé ---
#    Cette classe est la "plomberie" qui connecte
#    votre fichier .npz au DataLoader de PyTorch.
class CarRacingDataset(Dataset):
    def __init__(self, data_file):
        print(f"Chargement des données depuis {data_file}...")
        data = np.load(data_file)
        # On ne prend que les observations, pas les actions
        self.observations = data['observations']
        print(f"Données chargées. Shape: {self.observations.shape}")

        # Les images sont en (N, H, W, C). PyTorch (Conv2d)
        # veut (N, C, H, W). Nous devons permuter les axes.
        # (N, 64, 64, 3) -> (N, 3, 64, 64)
        self.observations_tensor = torch.from_numpy(self.observations).permute(0, 3, 1, 2)
        print("Données permutées pour PyTorch.")

    def __len__(self):
        return len(self.observations_tensor)

    def __getitem__(self, idx):
        # Le DataLoader s'occupera de créer les batchs
        return self.observations_tensor[idx]

In [ ]:
# --- 3. Le Modèle CVAE (identique à avant) ---
class CVAE(nn.Module):
    def __init__(self, z_dim, image_channels=3):
        super(CVAE, self).__init__()
        self.z_dim = z_dim

        # --- Encodeur (Image -> Espace Latent) ---
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.ReLU()
        )
        self.flat_size = 256 * 4 * 4
        self.fc_mu = nn.Linear(self.flat_size, z_dim)
        self.fc_logvar = nn.Linear(self.flat_size, z_dim)

        # --- Décodeur (Espace Latent -> Image) ---
        self.decoder_fc = nn.Linear(z_dim, self.flat_size)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, image_channels, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        h_flat = h.view(-1, self.flat_size)
        return self.fc_mu(h_flat), self.fc_logvar(h_flat)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.decoder_fc(z))
        h_unflat = h.view(-1, 256, 4, 4)
        return self.decoder(h_unflat)

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        recon_x = self.decode(z)
        return recon_x, mu, log_var

In [ ]:
# --- 4. Fonction de Perte (Loss) VAE (identique à avant) ---
def vae_loss_function(recon_x, x, mu, log_var):
    # Perte de Reconstruction (BCE). 'reduction="sum"' est important.
    recon_loss = F.binary_cross_entropy(recon_x, x, reduction='sum')

    # Perte de Régularisation (KLD)
    kld = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    return recon_loss + kld

In [ ]:
# --- 5. Script Principal d'Entraînement ---
if __name__ == "__main__":

    # 1. Détecter le GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Utilisation du device : {device}")

    # 2. Créer le Dataset et le DataLoader
    dataset = CarRacingDataset(DATA_FILE)
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2 # Utilise des sous-processus pour charger les données
    )

    # 3. Initialiser le Modèle et l'Optimiseur
    model = CVAE(z_dim, image_channels).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print("Début de l'entraînement du CVAE...")
    model.train() # Mettre le modèle en mode entraînement

    # 4. Boucle d'Entraînement
    for epoch in range(num_epochs):
        total_epoch_loss = 0

        # tqdm pour la barre de progression
        pbar = tqdm(data_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for images in pbar:
            images = images.to(device)

            # --- Forward pass ---
            recon_images, mu, log_var = model(images)

            # --- Calcul de la perte ---
            loss = vae_loss_function(recon_images, images, mu, log_var)

            # --- Backward pass ---
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_epoch_loss += loss.item()
            pbar.set_postfix(loss=f"{loss.item()/len(images):.4f}")

        # Calculer la perte moyenne pour l'époque
        avg_loss = total_epoch_loss / len(dataset)
        print(f"Fin Epoch {epoch+1}. Perte moyenne : {avg_loss:.4f}")

    # 5. Sauvegarder le modèle entraîné
    print("Entraînement terminé.")
    print(f"Sauvegarde du modèle dans {MODEL_SAVE_PATH}...")
    torch.save(model.state_dict(), MODEL_SAVE_PATH)
    print("Modèle sauvegardé !")